In [30]:
import os
import pandas as pd
import toml

In [166]:
# Specify config 
config_path = "../configs/config_2024.toml"
config = toml.load(config_path)

In [168]:
def process_results(county, analysis_year):  

    df_running = pd.read_csv(os.path.join(config["output_root"], f'data/county/{county}/{analysis_year}/running_summary.csv'))
    df_start = pd.read_csv(os.path.join(config["output_root"], f'data/county//{county}/{analysis_year}/start_summary.csv'))

    # Relabel vehicle types for clarity
    df_running['veh_type'] = df_running["veh_type"].map(config["veh_type_map"])
    df_start['veh_type'] = df_start["veh_type"].map(config["veh_type_map"])

    df_pollutant_running = df_running[df_running['pollutant_name'].isin(config["pollutant_list"])].groupby(['veh_type',"pollutant_name"]).sum()[['running_daily_tons']]
    df_pollutant_running.rename(columns={'running_daily_tons': analysis_year}, inplace=True)
    
    df_pollutant_start = df_start[df_start['pollutant_name'].isin(config["pollutant_list"])].groupby(['veh_type',"pollutant_name"]).sum()[['start_tons']]
    df_pollutant_start.rename(columns={'start_tons': analysis_year}, inplace=True)

    # VMT is the same for all pollutants; select the first one and report 
    df_vmt = df_running[df_running['pollutant_name']==config["pollutant_list"][0]].groupby(['veh_type']).sum()[['daily_vmt_total']]
    df_vmt.rename(columns={'daily_vmt_total': analysis_year}, inplace=True)

    return df_pollutant_running, df_pollutant_start, df_vmt

In [ ]:
county = "King"

running_df = pd.DataFrame()
start_df = pd.DataFrame()
vmt_df = pd.DataFrame()

for year in config["analysis_year_list"]:
    df_year_running_df, df_year_start_df, df_year_vmt_df = process_results(county, year)

    if len(running_df>0): 
        running_df = running_df.merge(df_year_running_df, on=["veh_type","pollutant_name"])
        start_df = start_df.merge(df_year_start_df, on=["veh_type","pollutant_name"])
    else:
        running_df = df_year_running_df.copy()
        start_df = df_year_start_df.copy()
        vmt_df = df_year_vmt_df.copy()

# Total Emissions
total_df = running_df.merge(start_df, on=["veh_type","pollutant_name"], suffixes=["_running", "_start"])
for year in config["analysis_year_list"]:
    total_df[year] = total_df[f"{year}_running"] + total_df[f"{year}_start"]
total_df = total_df[config["analysis_year_list"]]

# Sort by vehicle type order and pollutant name
veh_type_order = ["Passenger Vehicles", "Medium Trucks", "Heavy Trucks", "Bus"]
total_df = total_df.sort_values(by=["veh_type", "pollutant_name"], 
                                key=lambda x: x.map({v: i for i, v in enumerate(config["pollutant_list"])} if x.name == "pollutant_name" else {v: i for i, v in enumerate(veh_type_order)}))

with pd.ExcelWriter(os.path.join(config["output_root"],'county_summary_results.xlsx')) as writer:
    total_df.to_excel(writer, sheet_name='Total Emissions')
    running_df.to_excel(writer, sheet_name='Running Emissions')
    start_df.to_excel(writer, sheet_name='Start Emissions')
    vmt_df.to_excel(writer, sheet_name='VMT')

In [180]:
running_df

,veh_type,pollutant_name,2023,2024
6,Medium Trucks,CO2 Equivalent,1431.606221,1399.172099
9,Passenger Vehicles,CO2 Equivalent,16089.725522,15573.137541
3,Heavy Trucks,CO2 Equivalent,2719.099048,2682.676414
0,Bus,CO2 Equivalent,249.369753,248.558768
7,Medium Trucks,Methane,0.252782,0.241869
1,Bus,Methane,0.308143,0.309368
4,Heavy Trucks,Methane,0.153615,0.151292
10,Passenger Vehicles,Methane,0.481056,0.460194
5,Heavy Trucks,N20,0.276325,0.278539
8,Medium Trucks,N20,0.080228,0.078821


2023          2024
veh_type           pollutant_name                            
Bus                CO2 Equivalent    249.955919    249.139479
                   Methane             0.309674      0.310887
                   N20                 0.021007      0.021134
Heavy Trucks       CO2 Equivalent   2720.491615   2684.050782
                   Methane             0.157130      0.154786
                   N20                 0.276388      0.278599
Medium Trucks      CO2 Equivalent   1496.543066   1462.238045
                   Methane             0.420320      0.405022
                   N20                 0.116543      0.113610
Passenger Vehicles CO2 Equivalent  17185.586508  16623.133712
                   Methane             1.217180      1.161739
                   N20                 0.442097      0.423288

In [131]:
vmt_df

,2023,2024
veh_type,,
Bus,1.286701e+05,1.303171e+05
Heavy Trucks,1.418366e+06,1.436505e+06
Medium Trucks,1.957205e+06,1.982237e+06
Passenger Vehicles,4.108819e+07,4.160847e+07
